<a href="https://colab.research.google.com/github/ddoubleseat/MusicVAE/blob/main/Music_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Hierarchical Latent Vector Model for Learning Long-Term Structure in Music**

  기존의 VAE(Variational Autoencoder)를 개선 시킨것이 MusicVAE라고 할 수 있다. 그렇다면 Variational은 무엇이고 Autoencoder란 무엇일까? 우선 Autoencoder가 무엇인지 알아보자.

## Autoencoder vs Variational Autoencoder

  Autoencoder는 비지도 unsupervised 방식으로 훈련된 인공 신경망으로, 먼저 데이터에 인코딩 된 표현을 학습한 다음, 학습 된 인코딩 표현에서 입력 데이터를 (가능한한 가깝게) 생성하는 것을 목표로 한다. 따라서, 오토인코더의 출력은 입력에 대한 예측이다. 그렇다면 Variational Autoencoder란 무엇일까.  
AE와 VAE의 가장 큰 차이점은 AE는 Latent space z에 값을 저장하고 VAE는 확률분포를 저장하여 파라미터를 생성한다는 점이라고 한다. 즉 AE는 input x 자신을 언제든지 reconstruct할 수 있는 z를 만드는것이 목적이고 VAE는 input x가 만들어지는 확률 분포를 찾는것이 목적이다.

## MusicVAE / A Hierarchical Latent Vector Model for Learning Long-Term Structure in Music

VAE는 자연 데이터로부터 의미있는 결과를 도출하는 모델임이 분명했지만, sequential data적용에는 제한이 있었고 기존의 순환 VAE 모델은 음악과 같이 long-term structured sequence를 모델링하는데 어려움이 있었다고 한다. 첫번째로 RNN은 강력한 자기 회귀 모델이기 떄문에 RNN 디코더는 데이터를 학습하기 위해 Latent를 무시할 수 있기 때문이고 두번째는 Autoencoding은 전체 시퀀스를 하나의 Latent Vector로 압축해야 했기 때문에 짧은 시퀀스에는 잘 작동하지만 길이기 길어질 경우에는 성능이 현저히 떨어졌기 때문이다.  
따라서 이 문제를 해결하기 위해 MusicVAE에서는 Hierarchical RNN을 디코더에 사용한다. 이 구조는 모델이 Latent Code를 좀 더 활용하도록하여 recurrent VAE의 고질적인 문제인 'posterior collapse(사후 붕괴)'를 피할 수 있도록 한다.  
ResNet 모델에서 Vanishing Gradient 문제를 회피하기 위해 Residual Block 개념을 도입한것과 유사하다는 생각이 든다.

# Mission

## *Music VAE, Groove MIDI Dataset을 이용하여 4마디에 해당하는 드럼 샘플을 뽑아내는 과정을 수행*

## Mount Google Drive

In [1]:
# Colab 사용시
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install

In [2]:
# !apt-get update -qq && apt-get install -qq libfluidsynth2 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
# !pip install -q pyfluidsynth
# !pip install -U -q magenta

### Import Library

In [3]:
# 환경 설정
import os
import numpy as np
import pandas as pd
import collections

import tensorflow_datasets as tfds
import tensorflow.compat.v1 as tf
import tf_slim

# Colab/Notebook specific stuff
# import IPython.display
# from IPython.display import Audio
# from google.colab import files

# Magenta specific stuff
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import configs
from magenta.models.music_vae import data
from magenta.models.music_vae import data_hierarchical
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae.base_model import MusicVAE
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.scripts.convert_dir_to_note_sequences import convert_directory

import note_seq
from note_seq import midi_synth
from note_seq.sequences_lib import concatenate_sequences
from note_seq.protobuf import music_pb2

HParams = contrib_training.HParams

/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


### Hyper_Parameters

In [4]:
# Colab directory
DATA_DIR = '/content/drive/MyDrive/music_vae/Dataset/groove-v1.0.0-midionly/groove/'
TFRECORD_DIR = '/content/drive/MyDrive/music_vae/Tfrecord/music.tfrecord'
TRAIN_DIR = '/content/drive/MyDrive/music_vae/Train/'

# HParams
BATCH_SIZE=512 # batch size
MAX_SEQ_LEN=16 * 4  # 4 bars w/ 16 steps per bar, 시퀀스 길이 제한
Z_SIZE=256 # Latent Vector
ENC_RNN_SIZE=[512] # Encoder RNN SIZE
DEC_RNN_SIZE=[256, 256] # Decoder RNN SIZE
MAX_BETA=0.2
FREE_BITS=48
DROPOUT_KEEP_PROB=0.3 # Drop out

NUM_STEPS = 100 # Epoch

## 1. Preprocessing

In [5]:

convert_directory(DATA_DIR,TFRECORD_DIR,recursive=True)

## 2. Set Config

configuratoin 설정(From configs.py)

In [6]:
# Config 클래스
class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

  def values(self):
    return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)

def update_config(config, update_dict):
  config_dict = config.values()
  config_dict.update(update_dict)
  return Config(**config_dict)

CONFIG_MAP = {}

# GrooVAE config
# 드럼 4마디를 생성하기 위한 모델의 config
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()),
    # 3.Model
    # 3.1 Bidirectional Encoder - Encoder에 Bi-LSTM(양방향 LSTM)신경망 적용
    # RNN이나 LSTM은 입력 순서를 시간 순대로 입력하기 때문에 결과물이 직전 패턴을 기반으로 수렴하는 경향을 보인다는 한계가 있다.
    # 이 단점을 해결하는 목적으로 양방향 순한신경망(Bi-RNN)이 제안되었다. Bi-RNN은 기존의 순방향에 역방향을 추가하여,
    # 은닉층에 추가하여 성능을 향상시켰다. 그러나 데이터 길이가 길고 층이 깊으면, 과거의 정보가 손실되는 단점이 있다.
    # 이를 극복하기 위해 제안된 알고리즘이 양방향 LSTM이다

    # 3.2 Hierarchical Decoder - Decoder에 계층적 2-Layer LSTM 신경망 적용
    # 기존 Recurrent VAE의 decoder는 단순한 Stack RNN이다. 이 모델은 긴 Sequence에 대한 샘플링 결과가 좋지 않았고 원인은
    # Output Sequence가 생성됨에 따라 Latent State가 힘을 잃기기 때문이라고 유추된다. 따라서 단방향 LSTM 계층을 추가하여
    # 2계층 단방향 LSTM 신경망이 서로 종속적이고 계층적 구조를 갖게 하여 Latent State가 더 힘을 유지될 수 있게끔 해준다.

    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            # Hyper_Parameters에서 정의 - 모델의 파라미터
            batch_size=BATCH_SIZE,
            max_seq_len=MAX_SEQ_LEN,
            z_size=Z_SIZE,
            enc_rnn_size=ENC_RNN_SIZE,
            dec_rnn_size=DEC_RNN_SIZE,
            max_beta=MAX_BETA,
            free_bits=FREE_BITS,
            dropout_keep_prob=DROPOUT_KEEP_PROB,
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    tfds_name='groove/4bar-midionly',
)



## 3. Train Model

모델 학습(From music_vae_train.py)

In [7]:
# midi 데이터를 tensor로 변환
def _get_input_tensors(dataset, config):
  batch_size = config.hparams.batch_size
  iterator = tf.data.make_one_shot_iterator(dataset)
  (input_sequence, output_sequence, control_sequence,
   sequence_length) = iterator.get_next()
   # 데이터 로더
  input_sequence.set_shape(
      [batch_size, None, config.data_converter.input_depth])
  output_sequence.set_shape(
      [batch_size, None, config.data_converter.output_depth])
  if not config.data_converter.control_depth:
    control_sequence = None
  else:
    control_sequence.set_shape(
        [batch_size, None, config.data_converter.control_depth])
  sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

  return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
  }


# 학습
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
  """Train loop."""
  tf.gfile.MakeDirs(train_dir)
  is_chief = (task == 0)

  with tf.Graph().as_default():
    with tf.device(tf.train.replica_device_setter(
        num_ps_tasks, merge_devices=True)):

      model = config.model #config에서 설정한 모델 불러오기
      model.build(config.hparams, # 파라미터 설정
                  config.data_converter.output_depth,
                  is_training=True)

      optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

      hooks = []
      if num_sync_workers:
        optimizer = tf.train.SyncReplicasOptimizer(
            optimizer,
            num_sync_workers)
        hooks.append(optimizer.make_session_run_hook(is_chief))

      grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
      global_norm = tf.global_norm(grads)
      tf.summary.scalar('global_norm', global_norm)

      if config.hparams.clip_mode == 'value':
        g = config.hparams.grad_clip
        clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
      elif config.hparams.clip_mode == 'global_norm':
        clipped_grads = tf.cond(
            global_norm < config.hparams.grad_norm_clip_to_zero,
            lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                grads, config.hparams.grad_clip, use_norm=global_norm)[0],
            lambda: [tf.zeros(tf.shape(g)) for g in grads])
      else:
        raise ValueError(
            'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
      train_op = optimizer.apply_gradients(
          list(zip(clipped_grads, var_list)),
          global_step=model.global_step,
          name='train_step')

      logging_dict = {'global_step': model.global_step,
                      'loss': model.loss}

      hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
      if num_steps:
        hooks.append(tf.train.StopAtStepHook(last_step=num_steps))

      scaffold = tf.train.Scaffold(
          saver=tf.train.Saver(
              max_to_keep=checkpoints_to_keep,
              keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
      tf_slim.training.train(
          train_op=train_op,
          logdir=train_dir,
          scaffold=scaffold,
          hooks=hooks,
          save_checkpoint_secs=60,
          master=master,
          is_chief=is_chief)


def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator):
  """Load model params, save config file and start trainer.
  """
  config = config_map['groovae_4bar']
  is_training = True

  def dataset_fn():
    return data.get_dataset(
        config,
        tf_file_reader=tf_file_reader,
        is_training=True,
        cache_dataset=True)

  if is_training:
    train(
        train_dir = TRAIN_DIR,
        config=config,
        dataset_fn=dataset_fn,
        num_steps=NUM_STEPS)
  else:
    print("EVAL")

In [ ]:
run(CONFIG_MAP)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(
/usr/local/lib/python3.8/dist-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/usr/lo

## 3. Generate